# Investment crowdfunding has little faith in sustainability! At least for the moment.

## Journal:

*Venture Capital*, 25:1, 91-115, 2023, [DOI: 10.1080/13691066.2022.2129510](https://doi.org/10.1080/13691066.2022.2129510)

## Authors:

Carmen Mendoza

Isabel María Parra Oller

Álvaro Rezola (@alvarorezola)

Nuria Suárez

In [1]:
# Import libraries
import pandas as pd
import locale
import numpy as np
from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from sklearn.linear_model import LogisticRegression
from statsmodels.api import add_constant
from statsmodels.discrete.discrete_model import Probit
from statsmodels.discrete.discrete_model import Probit
import math
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Function definition
def significance_stars(pvalue):
    if pvalue < 0.01:
        return '***'
    elif pvalue < 0.05:
        return '**'
    elif pvalue < 0.1:
        return '*'
    else:
        return '-'
    
def summary_with_stars(model):
    # get the summary table as a DataFrame
    summary_df = model.summary2().tables[1]
    
    # create a new column for stars
    summary_df['stars'] = summary_df['P>|z|'].apply(significance_stars)

    # return the modified summary table
    return summary_df

def confidence_interval(X1, X2):
    #calculate degrees of freedom
    df = len(X1) + len(X2) - 2

    #calculate standard error
    s1 = np.var(X1, ddof=1)
    s2 = np.var(X2, ddof=1)
    n1 = len(X1)
    n2 = len(X2)
    standard_error = np.sqrt((s1/n1) + (s2/n2))

    #calculate margin of error
    margin_of_error = stats.t.ppf(0.95, df) * standard_error

    #calculate confidence interval
    lower_limit = (np.mean(X1) - np.mean(X2)) - margin_of_error
    upper_limit = (np.mean(X1) - np.mean(X2)) + margin_of_error

    return lower_limit, upper_limit

# calculamos el logit del puntuaje de propensión para emparejar
def logit(p):
    logit_value = math.log(p/(1 - p))
    return logit_value

# calcular standard errors ATE
def ate_se(df):
    # obtener el numero de observaciones de cada grupo
    n_t = df[df.sustainable == 1].shape[0]
    n_c = df[df.sustainable == 0].shape[0]
    
    # obtener la varianza de los resultados en cada grupo
    s_t = df[df.sustainable == 1].exito.var()
    s_c = df[df.sustainable == 0].exito.var()
    
    # calcular error estandar del ATE usando la fórmula
    se = np.sqrt((s_t / n_t) + (s_c / n_c))
    
    return se

In [3]:
# Import data
stata_dataset = "/home/alvaro/Desktop/MendozaEtAl2023-VC/Data/CROWD_SUSTAINABILITY_FINAL.dta"
df = pd.read_stata(stata_dataset)

# Prepare data-set
df.drop(index=range(3679,len(df)), inplace=True)
df = df[df["form_c"] == 1]
df["deadline"] = pd.to_datetime(df["deadline"])
df["datestart"] = pd.to_datetime(df["datestart"])
df["dateincorporation"] = pd.to_datetime(df["dateincorporation"])
df = df[df["deadline"] <= "2019-10-01"]
df["totalassetsmostrecent1"] += 1e-6
df["logtotalassetsmostrecent1"] = np.log(df["totalassetsmostrecent1"])
df = df.dropna(subset=["desastre6meses"])
df['desastre12meses_indemnizaciones'].fillna(0, inplace=True)
df["desastre12meses_indemnizaciones"] += 1e-6
df["log_desastre12meses_ind"] = np.log(df["desastre12meses_indemnizaciones"])

# Bank Net Income to Total Assets as value of the ratio net income-to-total assets in the banking sector per state and year.
df["bank_netincome_assets"] = df["lagnetinc"]/df["lagasset"]*100
df['bank_netincome_assets'].fillna(0, inplace=True)
df['banknetincome_assets_number'] = df['bank_netincome_assets']*df['lagbanks']
df['banknetincome_assets_number'].fillna(0, inplace=True)

# fill empty bank data
df['lagbranches'].fillna(0, inplace=True)
df['bank_netincome_assets'].fillna(0, inplace=True)
df['lagvcfundraising'].fillna(0, inplace=True)
df['loglagnum_oper_por_platf_y'].fillna(0, inplace=True)

# Treatment and control dataframes
df_sust = df[df["sustainable"] == 1]
df_non_sust = df[df["sustainable"] == 0]

# Fixed effect variables
year_dummies = ["yr2c"]
for i in range(3, 19):
    year_dummies.append(f"yr{i}c")
industry_dummies = ["ind2c"]
for i in range(3, 69):
    industry_dummies.append(f"ind{i}c")
state_dummies = ["state2c"]
for i in range(3, 63):
    state_dummies.append(f"state{i}c")

There are 1,766 investment crowdfunding campaings issued under the Form C exemption from May 1st, 2016 until September 10th, 2019.

### Table 1 (a): Descriptive statistics
This table shows the descriptive statistics – mean, standard deviation, 25th percentile, median, 75th percentile – of the main variables of interest.

In [4]:
df[["exito",
    "quick75relative",
    "sustainable",
    "totalassetsmostrecent1",
    "employees",
    "loglife",                     # Age as in np.log(date_diff(start, incorporation))              
    "equity",                     
    "asked",
    "lagbranches",                 # Bank branches
    "bank_netincome_assets",       # Ratio Net Income to Total Assets (after we will use banknetincome_assets_number)
    "lagvcfundraising",            # VC fundraising (not sure if then I need to do the np.log())
    "loglagnum_oper_por_platf_y",  # Number of offerings per platform
    ]].describe()


,exito,quick75relative,sustainable,totalassetsmostrecent1,employees,loglife,equity,asked,lagbranches,bank_netincome_assets,lagvcfundraising,loglagnum_oper_por_platf_y
count,1766.000000,1766.000000,1766.000000,1.766000e+03,1766.000000,1765.000000,1766.000000,1.766000e+03,1766.000000,1766.000000,1766.000000,1766.000000
mean,0.346546,0.026614,0.151755,3.346530e+05,5.361835,6.102006,0.270102,7.045705e+04,3954.537939,1.037196,7890.585938,67.921860
std,0.476001,0.160998,0.358885,1.762435e+06,10.314373,1.464374,0.444140,1.185048e+05,2366.321658,0.558752,10870.151367,65.648819
min,0.000000,0.000000,0.000000,1.000000e-06,0.000000,0.690000,0.000000,1.000000e+03,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000e-06,1.000000,5.320000,0.000000,1.000000e+04,1447.000000,0.818707,56.360001,9.000000
50%,0.000000,0.000000,0.000000,2.656100e+04,3.000000,6.350000,0.000000,2.500000e+04,4260.000000,0.885748,600.000000,37.000000
75%,1.000000,0.000000,0.000000,2.032070e+05,6.000000,7.150000,1.000000,7.875000e+04,6728.000000,1.091834,20763.099609,110.000000
max,1.000000,1.000000,1.000000,5.812937e+07,225.000000,9.870000,1.000000,1.070000e+06,6868.000000,4.925659,33174.179688,262.000000


 ### Table 1 (b): Mean differences across subsamples
 This table shows the mean values of the main variables across the two subsamples of offerings and the T-statistic for the mean differences. The T-statistics reported are obtained for the differences between the means across groups of offerings. All the variables are defined in Annex 1. ***, **, and * indicate statistical significance at 1, 5, and 10 percent, respectively.

In [5]:
data = {
    "success": (df_non_sust["exito"],df_sust["exito"]),
    "quick75relative": (df_non_sust["quick75relative"],df_sust["quick75relative"]),
    "totalassetsmostrecent1": (df_non_sust["totalassetsmostrecent1"],df_sust["totalassetsmostrecent1"]),
    "employees": (df_non_sust["employees"],df_sust["employees"]),
    "loglife": (df_non_sust["loglife"],df_sust["loglife"]),
    "equity": (df_non_sust["equity"],df_sust["equity"]),
    "asked": (df_non_sust["asked"],df_sust["asked"]),
    "lagbranches": (df_non_sust["lagbranches"],df_sust["lagbranches"]),
    "banknetincome_assets_number": (df_non_sust["banknetincome_assets_number"], df_sust["banknetincome_assets_number"]),
    "lagvcfundraising": (df_non_sust["lagvcfundraising"],df_sust["lagvcfundraising"]),
    "loglagnum_oper_por_platf_y": (df_non_sust["loglagnum_oper_por_platf_y"],df_sust["loglagnum_oper_por_platf_y"]),
}

results = {}

for key in data:
    group1 = data[key][0]
    group2 = data[key][1]

    # calculate t-statistic    
    t_statistic, p_value = stats.ttest_ind(group1, group2, nan_policy="omit")
    
    # Indicate statistical significance at different levels
    if p_value < 0.01:
        significance = "***"
    elif p_value < 0.05:
        significance = "**"
    elif p_value < 0.10:
        significance = "*"
    else:
        significance = "-"
        
    # calculate mean of each variable
    mean_group1 = np.mean(group1)
    mean_group2 = np.mean(group2)

    results[key]={'non sustainable':mean_group1,
                  'sustainable':mean_group2,
                  't-statistic':t_statistic,
                  'p-value':p_value,
                  'significance':significance}
# Display results
pd.DataFrame(results).T

,non sustainable,sustainable,t-statistic,p-value,significance
success,0.341121,0.376866,-1.132289,0.257667,-
quick75relative,0.02737,0.022388,0.46644,0.640958,-
totalassetsmostrecent1,327075.5,377007.5625,-0.427066,0.669383,-
employees,5.491989,4.634328,1.253924,0.210036,-
loglife,6.067301,6.295858,-2.356181,0.018573,**
equity,0.261682,0.317164,-1.884844,0.059615,*
asked,70123.804688,72319.789062,-0.279324,0.780029,-
lagbranches,4031.65287,3523.5,3.246528,0.00119,***
banknetincome_assets_number,139.827187,107.90303,3.804976,0.000147,***
lagvcfundraising,7952.545898,7544.255371,0.566209,0.571324,-


### Table 2: Propensity Score Matching 
This table shows the mean values of the main variables across the two subsamples of offerings and the t-statistics obtained for the differences between the means across groups of offerings, before matching and after matching using caliper, nearest 1-to-1 and nn-VBC methods. ***, ** and * indicate statistical significance at 1, 5, and 10%, respectively.

In [6]:
# Test for heteroskedascity to understand if we should use robust standard errors

In [7]:
# Two probit functions to validate covariates
formula_treatment = f"sustainable ~ logtotalassetsmostrecent1 + logemployees1 + logasked1"
treatment_model = smf.probit(formula=formula_treatment, data=df).fit(cov_type='HC0', disp=False)

# Get log-likelihood
log_likelihood = treatment_model.llf

# Get Wald chi-squared test statistic and p-value
wald_test = treatment_model.wald_test(treatment_model.model.exog_names)
wald_chi2 = wald_test.statistic[0][0]
wald_pvalue = wald_test.pvalue

print(f'Log-Likelihood: {log_likelihood:.4f}')
print(f'Wald Chi-Squared: {wald_chi2:.4f}')
print(f'Wald p-value: {wald_pvalue:.4f}')

summary_with_stars(treatment_model)

Log-Likelihood: -749.1635
Wald Chi-Squared: 801.8215
Wald p-value: 0.0000


,Coef.,Std.Err.,z,P>|z|,[0.025,0.975],stars
Intercept,-0.398424,0.355543,-1.120606,0.262456,-1.095275,0.298428,-
logtotalassetsmostrecent1,0.004643,0.003503,1.325460,0.185019,-0.002223,0.011509,-
logemployees1,-0.075966,0.045489,-1.669971,0.094925,-0.165123,0.013192,*
logasked1,-0.052412,0.033573,-1.561123,0.118495,-0.118213,0.013390,-


In [8]:
formula_outcome = f"exito ~ logtotalassetsmostrecent1 + logemployees1 + logasked1"
outcome_model = smf.probit(formula=formula_outcome, data=df).fit(cov_type='HC0', disp=False)

# Get log-likelihood
log_likelihood = outcome_model.llf

# Get Wald chi-squared test statistic and p-value
wald_test = outcome_model.wald_test(outcome_model.model.exog_names)
wald_chi2 = wald_test.statistic[0][0]
wald_pvalue = wald_test.pvalue

print(f'Log-Likelihood: {log_likelihood:.4f}')
print(f'Wald Chi-Squared: {wald_chi2:.4f}')
print(f'Wald p-value: {wald_pvalue:.4f}')

summary_with_stars(outcome_model)

Log-Likelihood: -1105.5368
Wald Chi-Squared: 227.5048
Wald p-value: 0.0000


,Coef.,Std.Err.,z,P>|z|,[0.025,0.975],stars
Intercept,1.497462,0.289416,5.174074,2.290439e-07,0.930216,2.064708,***
logtotalassetsmostrecent1,0.003466,0.003018,1.148521,2.507537e-01,-0.002449,0.009382,-
logemployees1,0.127192,0.039222,3.242845,1.183428e-03,0.050318,0.204066,***
logasked1,-0.201891,0.027189,-7.425398,1.124414e-13,-0.255181,-0.148601,***


### Table 2 (A): Before Matching

In [9]:
df[["exito", "quick75relative", "logtotalassetsmostrecent1", "logemployees1", "logasked1"]].describe()
df_sust = df[df["sustainable"] == 1]
df_non_sust = df[df["sustainable"] == 0]

data = {
    "success": (df_sust["exito"], df_non_sust["exito"]),
    "quick75relative": (df_sust["quick75relative"], df_non_sust["quick75relative"]),
    "logtotalassetsmostrecent1": (df_sust["logtotalassetsmostrecent1"], df_non_sust["logtotalassetsmostrecent1"]),
    "logemployees1": (df_sust["logemployees1"], df_non_sust["logemployees1"]),
    "logasked1": (df_sust["logasked1"], df_non_sust["logasked1"])
}

results = {}

for key in data:
    group1 = data[key][0]
    group2 = data[key][1]

    # calculate t-statistic    
    t_statistic, p_value = stats.ttest_ind(group1, group2, nan_policy="omit")
    
    # Indicate statistical significance at different levels
    if p_value < 0.01:
        significance = "***"
    elif p_value < 0.05:
        significance = "**"
    elif p_value < 0.10:
        significance = "*"
    else:
        significance = "-"
        
    # calculate mean of each variable
    mean_group1 = np.mean(group1)
    mean_group2 = np.mean(group2)

    results[key]={'sustainable':mean_group1,
                  'non_sustainable':mean_group2,
                  "difference" :(mean_group1 - mean_group2),
                  't-statistic':t_statistic,
                  'p-value':p_value,
                  'significance':significance}
# Display results
pd.DataFrame(results).T


,sustainable,non_sustainable,difference,t-statistic,p-value,significance
success,0.376866,0.341121,0.035744,1.132289,0.257667,-
quick75relative,0.022388,0.02737,-0.004982,-0.46644,0.640958,-
logtotalassetsmostrecent1,4.733572,4.211653,0.52192,0.699538,0.484308,-
logemployees1,1.36893,1.434692,-0.065762,-1.174444,0.240376,-
logasked1,10.326214,10.433923,-0.107709,-1.415024,0.157238,-


### Table 2 (B): After Matching

In [10]:
# Logit model para estimar el puntaje de propensión (ps)
model = LogisticRegression()
df = df.dropna(axis = 1) # drop all variables that have empty values

# Independent variables
X = df[["logtotalassetsmostrecent1",
        "logemployees1",
        "logasked1"]]

# Dependent variable (treatment group)
y = df["sustainable"]

# Model adjustment & predicted probabilities
model.fit(X, y) 
pred_prob = model.predict_proba(X)
df["ps"] = pred_prob[:, 1]

df["ps_logit"] = df.ps.apply(logit)

# Implementing the caliper match
def caliper_match(df, threshold):
    # ordenar los datos por ps_logit y crear una columna con el índice original
    df_sorted = df.sort_values("ps_logit").reset_index()
    df_sorted["orig_index"] = df_sorted.index
    
    # crear listas vacias para almacenar los indices emparejados y no emparejados
    matched_index = []
    unmatched_index = []
    
    # iterar sobre las filas del dataframe ordenado
    for i in range(len(df_sorted)):
        row = df_sorted.iloc[i]
        if i not in matched_index: # si la fila no está emparejada todavía
            potential_matches = df_sorted[(df_sorted.sustainable != row.sustainable) & (abs(df_sorted.ps_logit - row.ps_logit) <= threshold)]
            
            # encontrar las filas potenciales que tienen un tratamiento diferente y una diferencia de ps_logit menor o igual al umbral
            
            if len(potential_matches) > 0: # si hay al menos una fila potencialmente emparejable
                closest_match_index = potential_matches.iloc[0].orig_index
                # Tomar la priemra fila potencial como la más cercana
                
                matched_index.append(i)
                matched_index.append(closest_match_index)
                # añadir ambos índices a la lista de emparejados
                
            else:
                unmatched_index.append(i)
                # si no hay ninguna fila potencialmente emparejable, añadir el índice a la lista d eno emparejados
    return matched_index, unmatched_index

caliper_matched, caliper_unmatched = caliper_match(df[["sustainable", "logtotalassetsmostrecent1", "logemployees1", "logasked1", "ps_logit"]], 0.1) # umbral 0.2
caliper_df_matched = df.iloc[caliper_matched]

# mean of each variable in the treatment & control group
treatment_means_caliper = caliper_df_matched[caliper_df_matched["sustainable"] == 1]
control_means_caliper = caliper_df_matched[caliper_df_matched["sustainable"] == 0]

data = {
    "success": (treatment_means_caliper["exito"], control_means_caliper["exito"]),
    "quick75relative": (treatment_means_caliper["quick75relative"], control_means_caliper["quick75relative"]),
    "logtotalassetsmostrecent1": (treatment_means_caliper["logtotalassetsmostrecent1"], control_means_caliper["logtotalassetsmostrecent1"]),
    "logemployees1": (treatment_means_caliper["logemployees1"], control_means_caliper["logemployees1"]),
    "logasked1": (treatment_means_caliper["logasked1"], control_means_caliper["logasked1"])
}

results = {}

for key in data:
    group1 = data[key][0]
    group2 = data[key][1]

    # calculate t-statistic    
    t_statistic, p_value = stats.ttest_ind(group1, group2, nan_policy="omit")
    
    # Indicate statistical significance at different levels
    if p_value < 0.01:
        significance = "***"
    elif p_value < 0.05:
        significance = "**"
    elif p_value < 0.10:
        significance = "*"
    else:
        significance = "-"
        
    # calculate mean of each variable
    mean_group1 = np.mean(group1)
    mean_group2 = np.mean(group2)

    results[key]={'sustainable':mean_group1,
                  'non_sustainable':mean_group2,
                  "difference" :(mean_group1 - mean_group2),
                  't-statistic':t_statistic,
                  'p-value':p_value,
                  'significance':significance}
# Display results
pd.DataFrame(results).T

,sustainable,non_sustainable,difference,t-statistic,p-value,significance
success,0.340564,0.306615,0.033949,1.468236,0.142129,-
quick75relative,0.017354,0.020528,-0.003174,-0.45255,0.650901,-
logtotalassetsmostrecent1,5.069388,5.53946,-0.470072,-0.902258,0.366981,-
logemployees1,1.318171,1.482502,-0.16433,-4.057515,0.000051,***
logasked1,10.476681,10.427021,0.04966,0.890431,0.373295,-


### Table 3: Average Treatment Effect on the Treated (ATET) 
This table shows the average treatment effect on the treated individuals (ATET) for each method: caliper, nearest 1-to-1 and nn-VBC methods. ***, ** and * indicate statistical significance at 1, 5, and 10%, respectively.

In [11]:
caliper_ate_success = caliper_df_matched.groupby("sustainable")["exito"].mean().diff().iloc[-1]
caliper_ate_quick = caliper_df_matched.groupby("sustainable")["quick75relative"].mean().diff().iloc[-1]
caliper_se = ate_se(caliper_df_matched)

# realizar una preuba de t de dos muestras independientes y obtener los valores p y los intervalso de confianza al 95%
caliper_success_tstat, caliper_success_pvalue, caliper_success_desc = sms.ttest_ind(
    caliper_df_matched[caliper_df_matched.sustainable == 1].exito.values,
    caliper_df_matched[caliper_df_matched.sustainable == 0].exito.values,
    usevar="unequal",
    alternative="larger",
    value=0
)
lower_limit_success, upper_limit_success = confidence_interval(caliper_df_matched[caliper_df_matched.sustainable == 1].exito.values,
                                                               caliper_df_matched[caliper_df_matched.sustainable == 0].exito.values)

# realizar una preuba de t de dos muestras independientes y obtener los valores p y los intervalso de confianza al 95%
caliper_quick_tstat, caliper_quick_pvalue, caliper_quick_desc = sms.ttest_ind(
    caliper_df_matched[caliper_df_matched.sustainable == 1].quick75relative.values,
    caliper_df_matched[caliper_df_matched.sustainable == 0].quick75relative.values,
    usevar="unequal",
    alternative="larger",
    value=0
)

lower_limit_quick, upper_limit_quick = confidence_interval(caliper_df_matched[caliper_df_matched.sustainable == 1].quick75relative.values,
                                                           caliper_df_matched[caliper_df_matched.sustainable == 0].quick75relative.values)

data = {"Modelo caliper 1-to-1": ["success", "quickrelative75"],
        "ATE": [caliper_ate_success, caliper_ate_quick],
        "Error estándar": [caliper_se, caliper_se],
        "Valor p": [caliper_success_pvalue, caliper_quick_pvalue],
        "[Intervalo de": [lower_limit_success, lower_limit_quick],
        "confianza (95%)]": [upper_limit_success, upper_limit_quick]}

pd.DataFrame(data)

,Modelo caliper 1-to-1,ATE,Error estándar,Valor p,[Intervalo de,confianza (95%)]
0,success,0.033949,0.023612,0.075504,-0.004899,0.072798
1,quickrelative75,-0.003174,0.023612,0.684516,-0.014041,0.007693


### Table 4: Sustainability and success
This table presents IV results examining the effect of the sustainable orientation of investment crowdfunding offerings on the probability of success. The dependent variable in columns (1) and (2) is the dummy that identifies sustainable offerings (Sustainable). The dependent variable in columns (3) and (5) is SUCCESS. QUICK75 is the dependent variables in columns (4) and (6). Variables definitions are reported in Annex 1. Year, industry-year and state-year fixed effects are included but not reported. T-statistics are in parentheses. ***, ** and * indicate statistical significance at 1, 5, and 10%, respectively.

$$
SUCCESSFUL_{ijklt} = \beta_0 + \beta_1SUSTAINABLE\_predicted_{ijklt} + \beta_2FIRM_{jklt-1} + \beta_3OFFERING_{ijklt} + \theta_k + \delta_l + \lambda_t + \epsilon_{ijklt}
$$

In [17]:
# First-Stage Dependent Variable:
y = df['sustainable']

# specifications table 4.1
columns = ['desastre6meses',
           'sizemostrecent1',
           'logemployees1',
           'edad',
           'equity',
           'logasked1',
           "ind_year",
           "state_id"] + year_dummies
X1 = df[columns]

# Drop dummies that have a constant value for all rows
nunique = X1[year_dummies].nunique()
X1 = X1.drop(columns = nunique[nunique == 1].index)

# fit model 4.1
model1 = Probit(y, X1).fit(cov_type='HC0', disp=False)
results1 = pd.DataFrame(model1.params, columns=["Sustainable 1"])
results1["pvalue"] = model1.pvalues
results1.loc['Year Dummies', 'Sustainable 1'] = 'Yes'
results1.loc['Industry-Y Dummies', 'Sustainable 1'] = 'Yes'
results1.loc['State-Y Dummies', 'Sustainable 1'] = 'Yes'
results1['stars'] = results1["pvalue"].apply(significance_stars)
results1.loc['log_likelihood', 'Sustainable 1'] = model1.llf
wald_test = model1.wald_test(model1.model.exog_names)
wald_chi2 = wald_test.statistic[0][0]
results1.loc['wald_pvalue', 'Sustainable 1'] = wald_test.pvalue

# specifications table 4.2
columns = ['log_desastre12meses_ind',
           'sizemostrecent1',
           'logemployees1',
           'edad',
           'equity',
           'logasked1',
           "ind_year",
           "state_id"] + year_dummies

X2 = df[columns]

# Drop dummies that have a constant value for all rows
nunique = X2[year_dummies].nunique()
X2 = X2.drop(columns = nunique[nunique == 1].index)

# fit model 4.2
model42 = Probit(y, X2).fit(cov_type='HC0', disp=False)
results42 = pd.DataFrame(model42.params, columns=["Sustainable 2"])
results42["pvalue"] = model42.pvalues
results42.loc['Year Dummies', 'Sustainable 2'] = 'Yes'
results42.loc['Industry-Y Dummies', 'Sustainable 2'] = 'Yes'
results42.loc['State-Y Dummies', 'Sustainable 2'] = 'Yes'
results42['stars'] = results42["pvalue"].apply(significance_stars)
results42.loc['log_likelihood', 'Sustainable 2'] = model42.llf
wald_test = model42.wald_test(model42.model.exog_names)
wald_chi2 = wald_test.statistic[0][0]
results42.loc['wald_pvalue', 'Sustainable 2'] = wald_test.pvalue

# Desired order
row_order = ["desastre6meses",
             "log_desastre12meses_ind",
             "sizemostrecent1",
             "logemployees1",
             "edad",
             "equity",
             "logasked1",
             "ind_year",
             "state_id",
             'Year Dummies',
             "Industry-Y Dummies",
             "State-Y Dummies",
             "log_likelihood",
             "wald_pvalue"]

pd.concat([results1, results42], axis=1).reindex(row_order).drop(['ind_year', 'state_id'])


,Sustainable 1,pvalue,stars,Sustainable 2,pvalue,stars
desastre6meses,-0.292856,0.002204,***,NaN,NaN,NaN
log_desastre12meses_ind,NaN,NaN,NaN,-0.010479,0.007862,***
sizemostrecent1,-0.003814,0.674738,-,-0.004035,0.657251,-
logemployees1,-0.121083,0.011615,**,-0.118995,0.013181,**
edad,0.085368,0.007388,***,0.087022,0.006363,***
equity,0.17912,0.040536,**,0.17696,0.042525,**
logasked1,-0.039076,0.286853,-,-0.042908,0.239153,-
Year Dummies,Yes,NaN,-,Yes,NaN,-
Industry-Y Dummies,Yes,NaN,-,Yes,NaN,-
State-Y Dummies,Yes,NaN,-,Yes,NaN,-


In [13]:
# Uses the fitted model to make predictions for the dependent variable on both specifications (sustainable)
df['sustainablep_disasterdummy'] = model1.predict(X1)
df['sustainablep_disastercosts'] = model42.predict(X2)

# Specificación 1 (success)

# Without desastre6meses
controls = ["sizemostrecent1",
           'logemployees1',
           'edad',
           'equity',
           "logasked1",
           "ind_year",
           "state_id"] + year_dummies

Table_43_and_44_controls = controls + ["sustainablep_disasterdummy"]
Table_45_and_46_controls = controls + ["sustainablep_disastercosts"]

X2 = df[Table_43_and_44_controls]
X3 = df[Table_45_and_46_controls]

# Drop dummies that have a constant value for all rows
nunique_2 = X2[year_dummies].nunique()
X2 = X2.drop(columns = nunique_2[nunique_2 == 1].index)

nunique_3 = X3[year_dummies].nunique()
X3 = X3.drop(columns = nunique_3[nunique_3 == 1].index)

y2 = df['exito']
y3 = df["quick75relative"]

models = [Probit(y2, X2).fit(cov_type='HC0', disp=False),
          Probit(y3, X2).fit(cov_type='HC0', disp=False),
          Probit(y2, X3).fit(cov_type='HC0', disp=False),
          Probit(y3, X3).fit(cov_type='HC0', disp=False)]

results = []
for i, model in enumerate(models):
    result = pd.DataFrame(model.params, columns=[f"model{i+43}"])
    result["pvalue"] = model.pvalues
    result.loc['Observations', f"model{i+43}"] = model.nobs
    result.loc['Year Dummies', f"model{i+43}"] = 'Yes'
    result.loc['Industry-Y Dummies', f"model{i+43}"] = 'Yes'
    result.loc['State-Y Dummies', f"model{i+43}"] = 'Yes'
    result['stars'] = result["pvalue"].apply(significance_stars)
    result.loc['log_likelihood', f"model{i+43}"] = model.llf
    wald_test = model.wald_test(model.model.exog_names)
    wald_chi2 = wald_test.statistic[0][0]
    result.loc['wald_pvalue', f"model{i+43}"] = wald_test.pvalue
    if i < 2:
        result.loc['sustainable', f"model{i+43}"] = result.loc['sustainablep_disasterdummy', f"model{i+43}"]
        result.loc['sustainable', 'pvalue'] = result.loc['sustainablep_disasterdummy', 'pvalue']
        result.loc['sustainable', 'stars'] = result.loc['sustainablep_disasterdummy', 'stars']
    else:
        result.loc['sustainable', f"model{i+43}"] = result.loc['sustainablep_disastercosts', f"model{i+43}"]
        result.loc['sustainable', 'pvalue'] = result.loc['sustainablep_disastercosts', 'pvalue']
        result.loc['sustainable', 'stars'] = result.loc['sustainablep_disastercosts', 'stars']
    results.append(result)
    
# Desired order
row_order = ["desastre6meses",
             "log_desastre12meses_ind",
             "sustainable",
             "sizemostrecent1",
             "logemployees1",
             "edad",
             "equity",
             "logasked1",
             "ind_year",
             "state_id",
             'Year Dummies',
             "Industry-Y Dummies",
             "State-Y Dummies",
             "log_likelihood",
             "wald_pvalue",
             'Observations']

pd.concat(results, axis=1).reindex(row_order).drop(['ind_year', 'state_id'])


,model43,pvalue,stars,model44,pvalue,stars,model45,pvalue,stars,model46,pvalue,stars
desastre6meses,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
log_desastre12meses_ind,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sustainable,-2.480759,1.977161e-02,**,-1.512533,0.434449,-,-2.255703,6.503607e-02,*,-3.091069,0.128467,-
sizemostrecent1,-0.0041,6.084563e-01,-,0.003203,0.830318,-,-0.004054,6.124112e-01,-,0.002502,0.868745,-
logemployees1,0.064659,2.019437e-01,-,-0.29072,0.018840,**,0.070681,1.817241e-01,-,-0.33468,0.007626,***
edad,0.097497,4.156152e-03,***,-0.012783,0.841141,-,0.094053,8.163868e-03,***,0.016422,0.806779,-
equity,-0.123285,1.616579e-01,-,-0.088012,0.656417,-,-0.133864,1.412474e-01,-,-0.017784,0.924926,-
logasked1,-0.291324,6.331800e-20,***,-0.005686,0.930736,-,-0.28995,7.353723e-19,***,-0.022756,0.730066,-
Year Dummies,Yes,NaN,-,Yes,NaN,-,Yes,NaN,-,Yes,NaN,-
Industry-Y Dummies,Yes,NaN,-,Yes,NaN,-,Yes,NaN,-,Yes,NaN,-


### Table 5: Sustainability and success: the role of firm- and offering-level characteristics
This table presents results examining the effect of firm- and offering-level characteristics on the relationship between the sustainable orientation of investment crowdfunding offerings and the probability of success. The dependent variable is SUCCESS. Variables definitions are reported in Annex 1. Year, industry-year and state-year fixed effects are included but not reported. T-statistics are in parentheses. *** and ** indicate statistical significance at 1 and 5 percent, respectively.

$$
SUCCESSFUL_{ijklt} = \beta_0 + \beta_1SUSTAINABLE\_predicted_{ijklt} + \beta_2FIRM_{jklt-1} + \beta_3OFFERING_{ijklt} + \beta_4SUSTAINABLE\_predicted_{ijklt}*FIRM_{jklt-1} + \beta_5SUSTAINABLE\_predicted_{ijklt}*FIRM_{jklt-1}*OFFERING_{ijklt} + \theta_k + \delta_l + \lambda_t + \epsilon_{ijklt}
$$

In [14]:
interaction_cols = ['sizemostrecent1', 'logemployees1', 'edad', 'equity', 'logasked1']
for col in interaction_cols:
    df[f'sustainablep_{col}'] = df['sustainablep_disasterdummy'] * df[col]

# Without desastre6meses
controls = ['sustainablep_disasterdummy',
           'sizemostrecent1',
           'logemployees1',
           'edad',
           'equity',
           "logasked1",
           "ind_year",
           "state_id"] + year_dummies

models = []
for col in interaction_cols:
    X = df[controls + [f'sustainablep_{col}']]
    # Drop dummies that have a constant value for all rows
    nunique = X[year_dummies].nunique()
    X = X.drop(columns=nunique[nunique == 1].index)
    y = df['exito']
    model = Probit(y, X)
    models.append(model.fit(cov_type='HC0', disp=False))

results = []
for i, model in enumerate(models):
    result = pd.DataFrame(model.params, columns=[f"model{i+51}"])
    result["pvalue"] = model.pvalues
    result.loc['Observations', f"model{i+51}"] = model.nobs
    result.loc['Year Dummies', f"model{i+51}"] = 'Yes'
    result.loc['Industry-Y Dummies', f"model{i+51}"] = 'Yes'
    result.loc['State-Y Dummies', f"model{i+51}"] = 'Yes'
    result['stars'] = result["pvalue"].apply(significance_stars)
    result.loc['log_likelihood', f"model{i+51}"] = model.llf
    wald_test = model.wald_test(model.model.exog_names)
    wald_chi2 = wald_test.statistic[0][0]
    result.loc['wald_pvalue', f"model{i+51}"] = wald_test.pvalue
    results.append(result)
    
# Desired order
row_order = ["sustainablep_disasterdummy",
             "sustainablep_sizemostrecent1",
             "sustainablep_logemployees1",
             "sustainablep_edad",
             "sustainablep_equity",
             "sustainablep_logasked1",
             "sizemostrecent1",
             "logemployees1",
             "edad",
             "equity",
             "logasked1",
             "ind_year",
             "state_id",
             'Year Dummies',
             "Industry-Y Dummies",
             "State-Y Dummies",
             "log_likelihood",
             "wald_pvalue",
             'Observations']

pd.concat(results, axis=1).reindex(row_order).drop(['ind_year', 'state_id'])


,model51,pvalue,stars,model52,pvalue,stars,model53,pvalue,stars,model54,pvalue,stars,model55,pvalue,stars
sustainablep_disasterdummy,-3.047139,1.837588e-02,**,-3.285321,7.300449e-03,***,-5.830509,1.084529e-02,**,-3.243597,6.183154e-03,***,5.485633,0.195438,-
sustainablep_sizemostrecent1,0.06412,4.481038e-01,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sustainablep_logemployees1,NaN,NaN,NaN,0.673553,1.931233e-01,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sustainablep_edad,NaN,NaN,NaN,NaN,NaN,NaN,0.484448,9.803847e-02,*,NaN,NaN,NaN,NaN,NaN,NaN
sustainablep_equity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.666053,8.012313e-02,*,NaN,NaN,NaN
sustainablep_logasked1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.811517,0.054852,*
sizemostrecent1,-0.013637,3.692786e-01,-,-0.003941,6.227573e-01,-,-0.005272,5.128843e-01,-,-0.004249,5.953921e-01,-,-0.004479,0.576062,-
logemployees1,0.065069,1.996129e-01,-,-0.03159,7.236725e-01,-,0.059852,2.382809e-01,-,0.05728,2.642715e-01,-,0.056742,0.269484,-
edad,0.09775,4.068478e-03,***,0.095802,4.942942e-03,***,0.035725,4.702621e-01,-,0.101855,2.853778e-03,***,0.105022,0.002357,***
equity,-0.119912,1.735399e-01,-,-0.132619,1.336865e-01,-,-0.109974,2.124694e-01,-,-0.399297,2.542954e-02,**,-0.125912,0.155994,-


### Table 6: Sustainability and success: the role of the financing environment
This table presents results examining the effect of the characteristics of the financing environment on the relationship between the sustainable orientation of investment crowdfunding offerings and the probability of success. The dependent variable is SUCCESS. Variables definitions are reported in Annex 1. Firm and offering control variables, year, industry-year and state-year fixed effects are included but not reported. T-statistics are in parentheses. ***; ** and * indicate statistical significance at 1, 5, and 10 percent, respectively.

In [15]:
interaction_cols = ['lagbranches', 'bank_netincome_assets', 'lagvcfundraising', 'loglagnum_oper_por_platf_y']
for col in interaction_cols:
    df[f'sustainablep_{col}'] = df['sustainablep_disasterdummy'] * df[col]

# Without desastre6meses
controls = ['sustainablep_disasterdummy',
           'sizemostrecent1',
           'logemployees1',
           'edad',
           'equity',
           "logasked1",
           "ind_year",
           "state_id"] + year_dummies

models = []
for col in interaction_cols:
    X = df[controls + [f'sustainablep_{col}']]
    # Drop dummies that have a constant value for all rows
    nunique = X[year_dummies].nunique()
    X = X.drop(columns=nunique[nunique == 1].index)
    y = df['exito']
    model = Probit(y, X)
    models.append(model.fit(cov_type='HC0', disp=False))

results = []
for i, model in enumerate(models):
    result = pd.DataFrame(model.params, columns=[f"model{i+51}"])
    result["pvalue"] = model.pvalues
    result.loc['Observations', f"model{i+51}"] = model.nobs
    result.loc['Year Dummies', f"model{i+51}"] = 'Yes'
    result.loc['Industry-Y Dummies', f"model{i+51}"] = 'Yes'
    result.loc['State-Y Dummies', f"model{i+51}"] = 'Yes'
    result['stars'] = result["pvalue"].apply(significance_stars)
    result.loc['log_likelihood', f"model{i+51}"] = model.llf
    wald_test = model.wald_test(model.model.exog_names)
    wald_chi2 = wald_test.statistic[0][0]
    result.loc['wald_pvalue', f"model{i+51}"] = wald_test.pvalue
    results.append(result)
    
# Desired order
row_order = ["sustainablep_disasterdummy",
             "sustainablep_lagbranches",
             "sustainablep_bank_netincome_assets",
             'sustainablep_lagvcfundraising',
             "sustainablep_loglagnum_oper_por_platf_y",
             "sizemostrecent1",
             "logemployees1",
             "edad",
             "equity",
             "logasked1",
             "ind_year",
             "state_id",
             'Year Dummies',
             "Industry-Y Dummies",
             "State-Y Dummies",
             "log_likelihood",
             "wald_pvalue",
             'Observations']
pd.concat(results, axis=1).reindex(row_order).drop(['ind_year', 'state_id'])

,model51,pvalue,stars,model52,pvalue,stars,model53,pvalue,stars,model54,pvalue,stars
sustainablep_disasterdummy,-1.954556,7.859537e-02,*,-1.736998,1.515699e-01,-,-1.860464,8.784647e-02,*,-1.449679,1.863232e-01,-
sustainablep_lagbranches,0.000194,4.644861e-02,**,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sustainablep_bank_netincome_assets,NaN,NaN,NaN,-0.352835,1.960005e-01,-,NaN,NaN,NaN,NaN,NaN,NaN
sustainablep_lagvcfundraising,NaN,NaN,NaN,NaN,NaN,NaN,0.000084,6.387191e-04,***,NaN,NaN,NaN
sustainablep_loglagnum_oper_por_platf_y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.016398,2.521599e-05,***
sizemostrecent1,-0.00261,7.448124e-01,-,-0.003631,6.504685e-01,-,-0.002724,7.335104e-01,-,-0.005955,4.614628e-01,-
logemployees1,0.099114,6.730863e-02,*,0.076642,1.355736e-01,-,0.094856,6.729058e-02,*,0.101917,4.685037e-02,**
edad,0.071715,4.942253e-02,**,0.088155,1.120062e-02,**,0.069638,4.707028e-02,**,0.100936,3.337525e-03,***
equity,-0.166053,6.946000e-02,*,-0.138242,1.199774e-01,-,-0.164254,6.588370e-02,*,-0.125208,1.586581e-01,-
logasked1,-0.281503,3.372432e-18,***,-0.286413,4.543078e-19,***,-0.283971,9.784695e-19,***,-0.308899,1.564963e-21,***


### Table 7: Sustainability and success: robustness tests
This table presents a set of robustness tests for the relationship between the sustainable orientation of investment crowdfunding offerings and the probability of success. In column (1), we report the results for the second-stage regression for the Heckman (1979) method. In column (2), we find that the results do not vary when controlling for the characteristics of the team in terms of gender and size. In columns (3) to (5), we control for the funding history of the company. In column (6), we control for the cost structure defined by the funding portal. The dependent variable is SUCCESS. Variables definitions are reported in Annex 1. Firm and offering control variables, year, industry-year and state-year fixed effects are included but not reported. T-statistics are in parentheses. ***; ** and * indicate statistical significance at 1, 5, and 10 percent, respectively.